**This notebook covers all the code and operations performed during Ralstonia Solanacearum annotation project**

# Checking the quality of sequence : Fastq
# for downloading, i have created a conda environment "suraj_20".
# To make a environment:
conda create -n suraj_20
# Conda channels (a URL ) where conda look for packages.

#URLs
# defaults: 
[https://repo.anaconda.com/pkgs/main](https://repo.anaconda.com/pkgs/main)

#conda-forge:
[https://conda.anaconda.org/conda-forge](https://conda.anaconda.org/conda-forge)

#bioconda:
[https://conda.anaconda.org/bioconda](https://conda.anaconda.org/bioconda) 

#r:
[https://conda.anaconda.org/r](https://conda.anaconda.org/r) 

#anaconda:
[https://conda.anaconda.org/anaconda](https://conda.anaconda.org/anaconda) 

# 1. conda-forge
# 2. bioconda: for biological processes:
# ipykernel
# !Jupyterlab

In [ ]:
# Anaconda commands

#to add channel in base :
conda config --add channels bioconda

#List of configured channels:
conda config --show channels

#Remove channel:
conda config --remove channel_name

#create an environment:
conda create -n suraj_20

#List of existing environments:
conda env list

#to activate an environment:
conda activate suraj_20

#to deactivate and return to base:
conda deactivate

#Installing a tool:
conda install -c bioconda fastqc

# to remove an environment
conda remove --name ENV_NAME --all

In [ ]:
# Step 1
# To download fastqc
conda install -c bioconda fastqc
fastqc --help


In [ ]:
#fastqc command
!fastqc *.fastq -o ..Results/fastq


In [ ]:
# Step 2
# Filtering the data
# for that i have used trimmomatic: 

## Trimmomatic

Trimmomatic is a fast, flexible tool designed for trimming Illumina NGS data. It removes low-quality bases, adapter contamination, and short reads to improve downstream analysis like genome assembly or alignment. It supports both single-end and paired-end data.

In this pipeline, Trimmomatic was used to:
- Remove Illumina adapter sequences
- Trim low-quality bases from the read ends
- Discard very short reads

In [ ]:
# to install trimmomatic:
conda install -c bioconda trimmomatic
!trimmomatic --help

In [ ]:
trimmomatic PE G1_R1.fastq G1_R2.fastq \
../../Results/trimmomatic/G1_R1_paired.fastq.gz \
../../Results/trimmomatic/G1_R1_unpaired.fastq.gz \
../../Results/trimmomatic/G1_R2_paired.fastq.gz \
../../Results/trimmomatic/G1_R2_unpaired.fastq.gz \
SLIDINGWINDOW:4:20 MINLEN:25 ILLUMINACLIP:NexteraPE-PE.fa:2:40:15


### Command Explanation:
PE – Run mode: Paired-End

Input – G1_R1.fastq and G1_R2.fastq

Output (paired) – Cleaned paired reads

Output (unpaired) – Reads whose mates were discarded

SLIDINGWINDOW:4:20 – Trim when 4-base window average quality < 20

MINLEN:25 – Drop reads shorter than 25 bases

ILLUMINACLIP – Remove Nextera adapters using the adapter file

**Step 3**
**Genome Assembly**

## 🧬 What is Genome Assembly?

**Genome assembly** is the process of putting together short DNA sequences (called **reads**) generated by sequencing machines into longer sequences that represent the original genome.

Think of it like solving a jigsaw puzzle:
- Each read is a small puzzle piece.
- Assembly algorithms figure out how these pieces fit together based on overlapping regions.

### Why Is It Needed?

Sequencing machines (like Illumina) cannot read an entire genome in one go. Instead, they break it into millions of short reads, and genome assembly helps to:

- **Reconstruct the full genome** from these reads
- **Identify genes, mutations, and structure**
- **Enable annotation, comparison, and evolutionary analysis**

### Types of Assembly

**De novo** : Builds genome from reads **without any reference** 
**Reference-based** :  Aligns reads to a **known reference genome** 

In this project, we used **de novo assembly** with the tool **SPAdes** to assemble our Ralstonia genome without using a reference.


## 🧬 SPAdes Assembly – Run on Google Cloud Virtual Machine (VM)

Running **SPAdes** for de novo genome assembly is computationally intensive and can take several hours. It is not ideal for local systems due to high memory and CPU requirements. Therefore, I used a **Google Cloud Virtual Machine (VM)** to perform the assembly process.

---

### 🖥️ VM Configuration

I configured the VM with:
- **4 vCPUs**
- **16 GB RAM**

This machine type handled SPAdes efficiently. The entire assembly process took **around 8 hours**, which is expected for medium-sized bacterial genomes.

---

### SPAdes Installation (Download on VM)

Since SPAdes is not pre-installed, I first created a clean conda environment and then installed SPAdes using Bioconda:

In [ ]:
# Create a new conda environment named spades_env
conda create -n spades_env -c bioconda spades -y

# Activate the environment
conda activate spades_env

# Confirm installation
spades.py --version

In [ ]:
# After quality trimming with Trimmomatic, I ran the following command:
spades.py \
-1 ../../Results/trimmomatic/G1_R1_paired.fastq.gz \
-2 ../../Results/trimmomatic/G1_R2_paired.fastq.gz \
-o ../../Results/spades_assembly

In [ ]:
# I used tmux to prevent loss of progress in case of disconnection during the long SPAdes run:

tmux new -s spades_session

#After launching SPAdes inside the session, I detached using Ctrl + B then D. This allowed the job to continue running safely in the background even after I logged out.


**Spades Output files**: 

**contigs.fasta**: Contiguous sequences

**scaffolds.fasta**: Scaffolded sequences (using paired-end data)

**spades.log**: Log file of the assembly



**Step 4**

## 🧪 Quality Check of SPAdes Assembly

After completing the SPAdes assembly, I performed quality checks using two tools:

1. **QUAST** – to assess the structural quality of the assembly
2. **BUSCO** – to check the biological completeness using conserved single-copy genes

In [ ]:
# QUAST (Quality Assessment Tool): installation 

#To run QUAST smoothly, I created a dedicated Conda environment:

conda create -n quast_env -c bioconda quast -y
conda activate quast_env

QUAST helps evaluate how well the genome is assembled based on metrics like:

N50, L50: measure contiguity of contigs

Total length of the assembly

Number of contigs

GC content

Largest contig size

In [ ]:
#Quast command: 
quast.py \
./Results/spades_assembly/ragtag_scaffolded_GMI1000/ragtag.scaffold.fasta \
-o quast_ragtag_only_report_mac \
-l "RagTag Scaffolded Assembly" \
--min-contig 0


## 🧬 BUSCO – Biological Completeness Assessment

**BUSCO** (Benchmarking Universal Single-Copy Orthologs) is a powerful tool used to assess the **completeness** of a genome assembly or gene annotation. It checks for the presence of genes that are **highly conserved and expected to be present as single copies** in near-universal fashion across a given lineage.

In my project, BUSCO was used to validate how biologically complete the assembled Ralstonia genome is.

### 🧪 What BUSCO Reports

BUSCO evaluates assemblies and reports results under four categories:
- **Complete (C)** – genes found completely
  -  **Single-copy (S)** – found once as expected
  -  **Duplicated (D)** – found more than once
-  **Fragmented (F)** – partially found
-  **Missing (M)** – not found at all

These results give insight into the **quality, coverage, and completeness** of your genome.


###  BUSCO Installation (Docker)

Since BUSCO is complex and requires dependencies (e.g., Augustus, HMMER), I used the **official Docker container**, which is simple and portable.

In [ ]:
# Pull the BUSCO image (only once)
docker pull ezlabgva/busco:v5.8.2_cv1

# Run BUSCO using Docker
docker run \
  --platform linux/amd64 \
  -u $(id -u) \
  -v "$(pwd)":/busco_wd \
  ezlabgva/busco:v5.8.2_cv1 \
  busco \
  -i /busco_wd/Ralstonia_rathi_ragtag.fna \
  -o busco_results_for_ralstonia_ragtag \
  -m genome \
  -l bacteria_odb10 \
  -f


In [ ]:
# The way another than docker
# Create a dedicated environment
conda create -n busco_env -c bioconda -c conda-forge busco=5.4.3 -y
conda activate busco_env

# Download lineage dataset (e.g., for bacteria)
busco download --lineage bacteria_odb10


After running, BUSCO generates:

**short_summary.specific.bacteria_odb10.busco_results_for_ralstonia_ragtag.txt:** A quick summary with completeness %

**full_table.tsv:** Detailed gene-by-gene BUSCO matches

**run_*:** Folder with intermediate files, logs, and result visualizations

This quality check validated that my SPAdes assembly captured the expected bacterial core genes, ensuring that downstream functional annotation is meaningful.